In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/data/CiteSeer', name='CiteSeer')
print(len(dataset))
data = dataset[0]
data = data.to(device)
print(data)

cuda
1
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [2]:
import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn
from nni.retiarii import model_wrapper

In [3]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel_size=3, groups=in_ch)
        self.pointwise = nn.Conv2d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))


@model_wrapper
class ModelSpace(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # LayerChoice is used to select a layer between Conv2d and DwConv.
        self.conv2 = nn.LayerChoice([
            nn.Conv2d(32, 64, 3, 1),
            DepthwiseSeparableConv(32, 64)
        ])
        # ValueChoice is used to select a dropout rate.
        # ValueChoice can be used as parameter of modules wrapped in `nni.retiarii.nn.pytorch`
        # or customized modules wrapped with `@basic_unit`.
        self.dropout1 = nn.Dropout(nn.ValueChoice([0.25, 0.5, 0.75]))  # choose dropout rate from 0.25, 0.5 and 0.75
        self.dropout2 = nn.Dropout(0.5)
        feature = nn.ValueChoice([64, 128, 256])
        self.fc1 = nn.Linear(9216, feature)
        self.fc2 = nn.Linear(feature, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(self.conv2(x), 2)
        x = torch.flatten(self.dropout1(x), 1)
        x = self.fc2(self.dropout2(F.relu(self.fc1(x))))
        output = F.log_softmax(x, dim=1)
        return output

In [4]:
model_space = ModelSpace()
print(model_space)

ModelSpace(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): LayerChoice([Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1)), DepthwiseSeparableConv(
    (depthwise): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32)
    (pointwise): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
  )], label='model_1')
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


In [5]:
@model_wrapper
class MySpace(nn.Module):
    def __init__(self, num_node_features, num_classes, hidden=20):
        super().__init__()
        #self.conv1 = GCNConv(num_node_features, 16)
        #self.conv2 = GCNConv(16, num_classes)
        self.conv1 = nn.LayerChoice([
            nn.GATconv(num_node_features, hidden),
            nn.GCNConv(num_node_features, hidden),
            nn.GINconv(num_node_features, hidden),
            nn.ChebConv(num_node_features, hidden),
            nn.SAGEConv(num_node_features, hidden)
        ])
        self.dropout1 = nn.Dropout(nn.ValueChoice([0.1, 0.25, 0.5, 0.75]))  # choose dropout rate from 0.25, 0.5 and 0.75
        self.conv2 = nn.LayerChoice([
            nn.GATconv(hidden, num_classes),
            nn.GCNConv(hidden, num_classes),
            nn.GINconv(hidden, num_classes),
            nn.ChebConv(hidden, num_classes),
            nn.SAGEConv(hidden, num_classes)
        ])
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout1(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout2(x, training=self.training)
        x = F.softmax(x, dim=1)

        return x

In [6]:
my_space = MySpace(dataset.num_node_features, dataset.num_classes)
print(my_space)

AttributeError: module 'nni.retiarii.nn.pytorch' has no attribute 'GATconv'